In [1]:
%%bash
earthengine authenticate --quiet

Paste the following address into a web browser:

    https://accounts.google.com/o/oauth2/auth?redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&response_type=code&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control

On the web page, please authorize access to your Earth Engine account and copy the authentication code. Next authenticate with the following command:

    earthengine authenticate --authorization-code=PLACE_AUTH_CODE_HERE



In [2]:
%%bash
earthengine authenticate --authorization-code=


Successfully saved authorization token.


In [3]:
import ee

try:
  ee.Initialize()
  print('The Earth Engine package initialized successfully!')
except ee.EEException as e:
  print('The Earth Engine package failed to initialize!')
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

ee.Initialize()

The Earth Engine package initialized successfully!


In [4]:
import numpy as np
import time
import shutil

In [30]:
sensor = 'S2'#L8
res = 'mid'#hig,mid,ls

In [31]:
loc = 'bra'
if loc == 'bra':
    geometry = [[-44, 1],[-50, 1], [-50,-5], [-44,-5], [-44, 1]]
elif loc == 'bngldsh':
    geometry = [[87.802734375,26.745610382199025],
    [88.1103515625,21.08450008351735],
    [93.1640625,20.3034175184893],
    [92.7685546875,26.627818226393046],
    [87.802734375,26.745610382199025]]
elif loc == 'cmbd':
    geometry =[
            [
              102.216796875,
              14.455958231194037
            ],
            [
              102.50244140624999,
              10.35815140094367
            ],
            [
              104.87548828125,
              8.146242825034385
            ],
            [
              109.92919921875,
              11.609193407938953
            ],
            [
              109.6875,
              14.902321826141808
            ],
            [
              102.216796875,
              14.455958231194037
            ]
          ]

In [32]:
polygon = ee.Geometry.Polygon(geometry)

if sensor == 'S2':
    collection = ee.ImageCollection('COPERNICUS/S2')
elif sensor == 'L8':
    collection = ee.ImageCollection('LANDSAT/LC8_SR')#//LANDSAT/LC8_L1T_TOA_FMASK')
    
    def addMasked(image):
        FMASK_CLOUD = 4
        FMASK_CLOUD_SHADOW = 2
        fmask = image.select('cfmask')
        #cloudMask = fmask.neq(FMASK_CLOUD).and(fmask.neq(FMASK_CLOUD_SHADOW))
        cloudMask = fmask.neq(FMASK_CLOUD) and fmask.neq(FMASK_CLOUD_SHADOW)
        maskedimage = image.updateMask(cloudMask)
        return maskedimage    
    collection = collection.map(addMasked)
    
filtered = collection.filterDate('2016-05-01', '2016-11-30').filterBounds(polygon)
# 1.1-30.3.2017 for bngldsh, 10th percentile
# 2016-05-01', '2016-11-30, 5th percentile for brazil

prc = 5
perc = collection.reduce(ee.Reducer.percentile([prc]))
prc = str(prc)

if res == 'hig':
    scl = 10
    result = perc.select('B4_p'+prc, 'B3_p'+prc, 'B2_p'+prc, 'B8_p'+prc)
elif res == 'mid':
    scl = 20
    result = perc.select('B5_p'+prc, 'B6_p'+prc, 'B7_p'+prc)
elif res == 'ls':
    scl = 30
    result = perc.select('B2_p'+prc, 'B3_p'+prc, 'B4_p'+prc, 'B5_p'+prc, 'B6_p'+prc, 'B7_p'+prc)
    
task = ee.batch.Export.image.toCloudStorage(result, fileNamePrefix=loc+'_'+res+'_', bucket='testbucket_ron', scale=scl, maxPixels=99524201457, region=geometry)

In [33]:
task.start()